In [1]:
import pandas as pd
import numpy as np
import torch
!pip install fastai==0.7.0
!pip install torch
import fastai
from fastai.imports import *
from fastai.torch_imports import *
from fastai.core import *
from fastai.model import fit
from fastai.dataset import *\

from sklearn.svm import LinearSVC

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *
from fastai.text import *

import dill as pickle
!pip install spacy
!python -m spacy download en
import spacy
import html

import pickle
import collections
#!pip install nbimporter
import nbimporter

symbolic link created for C:\Users\lenovo\Anaconda3\envs\fastai-cpu\lib\site-packages\spacy\data\en <<===>> C:\Users\lenovo\Anaconda3\envs\fastai-cpu\lib\site-packages\en_core_web_sm

    Linking successful
    C:\Users\lenovo\Anaconda3\envs\fastai-cpu\lib\site-packages\en_core_web_sm
    -->
    C:\Users\lenovo\Anaconda3\envs\fastai-cpu\lib\site-packages\spacy\data\en

    You can now load the model via spacy.load('en')



In [2]:
#Das kommt ganz am Anfang, mit kleinem Text dass wir um den Code schlank zu halten einige Teile in Funktionen auslagern
import functions as fct

Importing Jupyter notebook from functions.ipynb


In [3]:
with open('wt103/itos_wt103.pkl', 'rb') as pickle_file:
    itos_wiki = pickle.load(pickle_file)
    
stoi_wiki = collections.defaultdict(lambda:-1, {v:k for k,v 
                                              in enumerate(itos_wiki)})

In [4]:
from functions import Prep

sent = 'Yesterday I flew with a new airline and it was great'
tok_sent = Prep.tokenize(sent, stoi_wiki)
print(tok_sent)

0
{'Tokens': ['\n', 'xbos', 'yesterday', 'i', 'flew', 'with', 'a', 'new', 'airline', 'and', 'it', 'was', 'great'], 'Encoded_Tokens': [288, 303, 13256, 74, 3273, 19, 9, 58, 4745, 6, 29, 12, 284]}


In [5]:
onehot = Prep.OneHot(tok_sent['Encoded_Tokens'], dimension = len(itos_wiki))
print(onehot)
onehot.shape

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


(13, 238462)

In [6]:
lm_wgts_wiki = torch.load('wt103/fwd_wt103.h5', map_location=lambda storage, 
                  loc: storage)

embedding = np.matmul(onehot, lm_wgts_wiki['0.encoder.weight'])
print(embedding)
embedding.shape

[[-0.4396  -0.39229 -0.12621 ...  0.12117  0.2433   0.10483]
 [-0.16437 -0.35955 -0.04734 ...  0.04385  1.08976 -0.09999]
 [ 0.28111 -0.59665  0.1551  ... -0.10522  0.31061  0.24847]
 ...
 [-0.10496  0.75285 -0.14176 ...  0.06853  0.02307  0.0305 ]
 [ 0.03717  0.86379 -0.01512 ... -0.01448 -0.64909 -0.00839]
 [ 0.5198   0.04776 -0.30437 ... -0.00925 -0.40007 -0.13416]]


(13, 400)

In [7]:
from functions import LSTM
lstm  = LSTM(embedding, lm_wgts_wiki, 1150)
st_lstm = LSTM.stacked(lstm)
print(st_lstm.hidden_state_l2)
st_lstm.hidden_state_l2.shape

[[-0.03709  0.19475 -0.01835 ... -0.00458  0.13194  0.01884]
 [-0.20202  0.13205 -0.01257 ... -0.01585  0.15082  0.10082]
 [-0.28976  0.1191  -0.10623 ... -0.10468 -0.30108  0.51579]
 ...
 [ 0.05157  0.11202  0.00307 ... -0.39102 -0.0455  -0.00869]
 [-0.0446   0.05912 -0.02197 ... -0.37411  0.04055 -0.33509]
 [-0.26005 -0.05814  0.1019  ... -0.48525 -0.17434  0.03801]]


(13, 400)

In [8]:
from functions import Prediction
prediction = Prediction.predict(st_lstm.hidden_state_l2, lm_wgts_wiki, stoi_wiki, tok_sent, 10)
print(prediction[0], "\n", prediction[1])

['\n xbos yesterday i flew with a new airline and it was great'] 
 [           word  probability
880     britain     0.000032
14            -     0.000005
9             a     0.000002
142    american     0.000002
490     america     0.000002
3             ,     0.000001
416     western     0.000001
537     success     0.000001
333        park     0.000001
7006  distances     0.000001]


In [9]:
# loading class Classifier from functions notebook
from functions import Classifier

# loading the twitter dataset vocabulary
with open('lm/itos.pkl', 'rb') as pickle_file:
    itos = pickle.load(pickle_file)

stoi = collections.defaultdict(lambda:0, 
                               {v:k for k,v in enumerate(itos)})

# loading the training model parameters of the classifier
clas_wgts = torch.load('models/clas_2.h5', map_location=lambda storage, 
                  loc: storage)

# prep functions executes tokenization, one-hot encoding, embedding
# and lstm forward pass, outputs hidden state tensor of last lstm
hidden_state = Classifier.prep('Yesterday I flew with a new airline and it was well', clas_wgts, stoi)
print(hidden_state)
hidden_state.shape

0
[[-0.00976  0.07716 -0.01014 ... -0.00183  0.02177 -0.00235]
 [-0.04179  0.13705 -0.01995 ... -0.00572  0.01427  0.00801]
 [-0.00527  0.04493 -0.04583 ... -0.02784 -0.04894  0.04374]
 ...
 [-0.06973  0.06097 -0.15617 ... -0.17291 -0.06177 -0.0094 ]
 [-0.04268  0.05418 -0.20595 ... -0.19399  0.02161 -0.23299]
 [ 0.0392   0.03127 -0.40149 ... -0.21588 -0.03951  0.02949]]


(13, 400)

In [10]:
# apply concat pooling for last hidden state vector (representing last word of tweet)
concat = Classifier.concat_pooling(hidden_state)
print(concat)
concat.shape

[ 0.0392   0.03127 -0.40149 ... -0.15368 -0.01226 -0.01494]


(1200,)

In [11]:
# process concat pooled hidden state in relu layer
relu = Classifier.relu_layer(concat, clas_wgts)
print(relu)
relu.shape

[[0.93098 0.      0.      0.20191 0.      0.      0.64166 0.      0.05019 0.57686 0.03932 0.68158 0.54921
  0.      0.39284 0.46535 0.29358 1.41082 0.08755 0.7847  0.63262 0.13294 0.      0.1568  0.57392 0.79917
  0.22868 0.40817 0.19757 0.53938 0.12102 0.      0.      0.      0.37147 0.      0.      0.      0.83357
  0.      0.      0.      0.12554 0.      0.417   0.      0.      0.      0.04186 0.25109]]


(1, 50)

In [14]:
# process output from relu layer in softmax layer and print out prediction
prediction = Classifier.clas_predict(relu, clas_wgts)
print(prediction)

{'negative': 0.24262342861838856, 'neutral': 0.047101041484506764, 'positive': 0.7102755298971046}
